In [1]:
from numba import njit
import numpy as np

@njit
def run_ising2d(N, T, J, B, n_steps, out_freq):
    '''Basic Metropolis Monte Carlo simulator of 2D Ising model
    ---
    spins:    (int, int) 2D numpy array
    T, J, B:  (floats) corresponding to temperature, coupling and field variables
    n_steps:  (int), simulation steeps
    out_freq: (int), How often to compute and save data
    ---
    Returns:
    E/N: per-spin energy over n steps
    M/N: per-spin magnetization over n steps
    S:   2D spin configurations over n steps
    '''

    #### Initialize spin-lattice configuration
    spins = 2*(np.random.rand(N, N) < 0.5) - 1

    # We use explicit looping required by njit accelearation
    M_t   = np.sum(spins)
    E_t   = -B*M_t
    for i in range(N):
        for j in range(N):
            z    = spins[(i+1)%N, j] + spins[(i-1)%N, j] + spins[i,(j+1)%N] + spins[i,(j-1)%N]
            E_t += -1/2 * J * z

    #### Run MC Simulation
    S, E, M = [], [], []
    for step in range(n_steps):

        # Pick random spin
        i, j = np.random.randint(N), np.random.randint(N)

        # Compute energy change resulting from a flip of spin at i,j
        z  = spins[(i+1)%N, j] + spins[(i-1)%N, j] + spins[i, (j+1)%N] + spins[i, (j-1)%N]
        dE = 2*spins[i,j]*(J*z + B)
        dM = 2*spins[i,j]

        # Metropolis condition
        if np.exp(-dE/T) > np.random.rand():
            spins[i,j] *= -1
            E_t        += dE
            M_t        += dM

        # Save Thermo data
        if step % out_freq == 0:
            M.append(M_t/N**2)
            E.append(E_t/N**2)
            S.append(spins.copy())

    return S, E, M

In [ ]:
#Simulation Parameters
params = {'N':40,
          'J':1,
          'B':0,
          'T': 4,
          'n_steps': 100000,
          'out_freq': 10}

# Simulation
S, E, M = run_ising2d(**params)

In [ ]:
np.array(S).shape, np.array(E).shape, np.array(M).shape

In [ ]:
@widgets.interact(i=(0, 10000-1))
def plot_image(i=0):

    fig, ax  = plt.subplots(ncols=3, figsize=(10,4))

    ax[0].pcolor(S[i])
    ax[1].plot(E)
    ax[2].plot(M)

    ax[0].set(ylabel='$i$', xlabel='j')
    ax[1].set(ylabel='$E$', xlabel='steps')
    ax[2].set(ylabel='$M$', xlabel='steps')
    fig.tight_layout()
    plt.show()

In [ ]:
#Simulation Parameters
params = {'N':40,
          'J':1,
          'B':0,
          'T': 4,
          'n_steps': 1000000,
          'out_freq': 10}

Ts = np.linspace(1, 4, 40)
Es, Ms, Cs, Xs = [], [], [], []

for T in Ts:

    params['T']=T

    S, E, M = run_ising2d(**params)

    # Save last 90% percent of data
    idx = int(len(E) * 0.1)

    E = E[idx:]
    M = M[idx:]

    Es.append(np.mean(E))
    Ms.append(np.mean(M))

    Cs.append(np.var(E)/T**2)
    Xs.append(np.var(M)/T)

In [ ]:
fig, ax  = plt.subplots(ncols=2, nrows=2, figsize=(8,6))

ax[0,0].scatter(Ts, Es)
ax[0,0].set(ylabel='$E(T)$')

ax[0,1].scatter(Ts, Ms)
ax[0,1].set(ylabel='$M(T)$')

ax[1,0].scatter(Ts, Cs)
ax[1,0].set(ylabel='$C_v(T)$')

ax[1,1].scatter(Ts, Xs)
ax[1,1].set(ylabel='$\Xi(T)$')
fig.tight_layout()

In [ ]:
def autocorr(x):
    n = len(x)
    variance = x.var()
    x = x - x.mean()
    result = np.correlate(x, x, mode='full')[-n:]
    result /= variance * np.arange(n, 0, -1)
    return result

M = np.random.randn(1000)  # Example data; replace with your magnetization array
acorr = autocorr(M)
plt.plot(acorr/acorr[0])
plt.xlabel('Steps, $n$')
plt.ylabel(r'$\langle M(t)M(t+n) \rangle$')
plt.show()